In [33]:
import sys
import time
import requests
from datetime import datetime, timedelta
import json
import fasttext
import re
from bs4 import BeautifulSoup
import lxml
import numpy as np
import os
import unidecode 
import pandas as pd
import pickle

EARLIEST_DATE = datetime(2019, 12, 19)
DIRNAME = 'baseline_state_analysis/'
# OUTPUT_DIRNAME = 'state_by_date/'

In [49]:
x = os.listdir(DIRNAME)
len(x)

5733

# GET ALL STATE DATA IN ONE PLACE

In [2]:
reddit_url = 'https://www.reddit.com/r/LocationReddits/wiki/faq/northamerica'
page = requests.get(reddit_url, headers = {'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(page.text, "lxml")
states = html.find_all("h5")[13:]
state_sub_map = {}
for state in states:
    if state.text == 'California':
        state_sub_map[state.text] = ['california']
        continue
    links = state.findNext().find_all('a')
    links = [link['href'][3:] for link in links]
    state_sub_map[state.text] = links
cali = html.find_all("h6")
for part in cali:
    links = part.findNext().find_all('a')
    links = [link['href'][3:] for link in links]
    state_sub_map['California'].extend(links)

In [3]:
def preproc(element):
    if 'body' in element.keys():
        text_tag = 'body'
    else:
        text_tag = 'selftext'
    line = element[text_tag]
    line = unidecode.unidecode(line)
    line = re.sub(r"&gt;.*?(\n|\s\s|\Z)", "", line)
    line = re.sub(r'\s\(\s', " ", line)
    line = re.sub(r"\n", " ", line)
    line = re.sub("\su/[a-zA-Z0-9-_]*(\s|\Z)", " USERNAME ", line)
    line = re.sub("\sr/[a-zA-Z0-9-_]*(\s|\Z)", " REDDIT ", line)    
    line = re.sub("\s{2,}", " ", line)
    element[text_tag] = line
    return element

In [4]:
# test = "Nothing about this is Alabama-specific. Post it r/coronavirus to r/coronavirus"
# # test = "Editorial cartoon by Bill Bramhall, May 11, 2020.\n\n[Link](https://www.nydailynews.com/opinion/ny-bramhall-editorial-cartoons-2020-20200107-7ctvclbkyng5nl3ocyeet2cax4-photogallery.html)\n\nThe coronacoaster continues  🎢 \n\n[GoLocalProv](https://www.golocalprov.com/news/19-new-ri-coronavirus-deaths-highest-in-5-days-watch-saturday-1-pm-briefing)link from 5/9/2020\n\n&gt;On Saturday, Rhode Island Governor lifted the stay at home order in effect in Rhode Island. \n\n**EDITOR'S NOTE: Raimondo does not allow Rhode Islanders to hear reporters' questions. Nearly all governors across the country and the White House allow reporters to ask questions directly and for the public to hear the questions and the responses.**\n\nFrom [WPRI](https://www.wpri.com/health/coronavirus/may-11-ri-coronavirus-update/) \n\nTakeaways: \n\n&gt;There were 176 new positive cases reported Monday, about **8%** of the 2,119 total test results returned.\n\n&gt;”I only want to reopen this economy once,” Raimondo said. ” I don’t want this to be fits and starts.”\n\nYeah, ok Gina. This isn’t about what *you* want. \n\n&gt;**Reporters were allowed back in the room to ask questions for the first time in nearly two months on Monday.** The location of the briefing was moved to the Veterans Memorial Auditorium, which sits next to the R.I. State House, to allow for more spacing."
# # print(test)
# output = preproc(test)
# print(output)

In [5]:
state_sub_map

{'Alabama': ['alabama',
  'Anniston',
  'ArabAlabama',
  'auburn',
  'baldwincountyal',
  'birmingham',
  'blountsville',
  'Calera',
  'CalhounCounty',
  'Cullman',
  'dothan',
  'DothanAL',
  'fairhopeAlabama',
  'FayetteCounty',
  'florenceal',
  'gadsden',
  'gordo',
  'greenvilleal',
  'Hartselle',
  'cityofhoover',
  'HooverAL',
  'huntsvillealabama',
  'JacksonvilleAL',
  'Madisonalabama',
  'marshallcountyal',
  'mobileal',
  'montgomery',
  'mountainbrook',
  'oba',
  'OxfordAL',
  'prattville',
  'Ragland',
  'selma',
  'shoals',
  '251',
  'SummerdaleAL',
  'sylacauga',
  'talladega',
  'ThomasvilleAL',
  'Trussville',
  'tuscaloosa',
  'vestavia',
  'vestaviahills'],
 'Alaska': ['alaska',
  'AleutianIslands',
  'Anchorage',
  'BarrowAK',
  'BethelAK',
  'Cordova',
  'fairbanks',
  'homerAK',
  'juneau',
  'Kenai',
  'ketchikan',
  'kodiak',
  'diomede',
  'MatSu',
  'nome',
  'PalmerAK',
  'Seward',
  'Sitka',
  'skagway',
  'spenardak',
  'sterlingalaska',
  'Talkeetna',
 

In [10]:
# state_data_counts = {}
for state in state_sub_map:
    state_data = []
    for sub in state_sub_map[state]:
        print(sub)
        if sub[-1] == "/":
            sub = sub[:-1]
        with open(DIRNAME + f'r.{sub}.submission.json', 'r') as file:
            state_data.extend([preproc(json.loads(line)) for line in file.readlines() if line != ''])
        with open(DIRNAME + f'r.{sub}.comment.json', 'r') as file:
            state_data.extend([preproc(json.loads(line)) for line in file.readlines() if line != ''])
#     state_data_counts[state] = len(state_data)
    with open(DIRNAME + f"{state}.all_data.json", "w", encoding="utf-8") as file:
        for line in state_data:
            json.dump(line, file)
            file.write("\n")

alabama
Anniston
ArabAlabama
auburn
baldwincountyal
birmingham
blountsville
Calera
CalhounCounty
Cullman
dothan
DothanAL
fairhopeAlabama
FayetteCounty
florenceal
gadsden
gordo
greenvilleal
Hartselle
cityofhoover
HooverAL
huntsvillealabama
JacksonvilleAL
Madisonalabama
marshallcountyal
mobileal
montgomery
mountainbrook
oba
OxfordAL
prattville
Ragland
selma
shoals
251
SummerdaleAL
sylacauga
talladega
ThomasvilleAL
Trussville
tuscaloosa
vestavia
vestaviahills
alaska
AleutianIslands
Anchorage
BarrowAK
BethelAK
Cordova
fairbanks
homerAK
juneau
Kenai
ketchikan
kodiak
diomede
MatSu
nome
PalmerAK
Seward
Sitka
skagway
spenardak
sterlingalaska
Talkeetna
valdez
Wasilla
arizona
ahwatukeeAZ
anthem
avondaleaz
Bagdad
bisbee
casagrandearizona
cavecreek
chandleraz
chinle
cochise
flagstaff
fountainhills
Gilbert
GilbertArizona
GlendaleAZ
grandcanyon
GoodyearAZ
happyjack
heberovergaard
jeromeaz
kingman
lakehavasucity
LakeHavasuCityAZ
LitchfieldPark
Marana
maricopa
mesaaz
mohavecounty
NewRiver
Nogales
nort

pinellas
PlantCity
polk
863
pontevedra
portcharlotte
portorange
portstjoe
portstjohn
PortStLucie
riverside
RockledgeFL
Sanford
SanfordFL
sanibel
sarasota
seaside
sebring
springhillfla
south_florida
southflorida
southwalton
southwood
staugustine
staugustinebeach
stjohnscounty
stpetersburgfl
swfl
SunnyIslesBeach
tampa
tampabay
tallahassee
tlh
tequesta
ThorntonPark
titusville
772
treasurecoast
valrico
veniceflorida
Venice_FL
WellingtonFL
WestPalmBeachFl
WinterPark
georgia
georgiausa
acworth
albanyga
alpharetta
athens
atlanta
Auburn_GA
augusta
barrowcounty
Brookhaven
CamdenGA
cantonga
carrolltongeorgia
cartersville
cherokeecountyga
CobbCounty
Cochran
columbiacounty
columbusga
conyers
cumming
CummingGA
dacula
dahlonega
daltonga
Dawsonville
decaturga
dvillega
dublinGA
DuluthGA
Dunwoody
snobbery
eastpoint
Ellijay
fayco
FalconCrest
flowerybranch
forsyth
franklincountyga
grovetown
gwinnett
Henrycounty
HinesvilleGA
hollyspringsga
jasperga
jekyllisland
johnscreek
kennesaw
lagrange
leesburgga
lilb

pontiacmi
posen
rochesterhills
romeomi
royaloak
saginaw
SaginawBay
SaginawMichigan
saintclairshores
shelbytwp
SouthBend
southwestmichigan
stclairshores
St_Joseph
swmich
swartzcreek
traversecity
uppermichigan
UPmichigan
washtenawannarbor
westmi
westernlowermichigan
WhiteLake
Whitehall
Woodhaven
woodhavenmi
wyomingmi
yooper
ypsilanti
ypsi
minnesota
alexandriamn
anokamn
applevalley
armatage
bemidji
bloomingtonmn
brainerd
buffalominnesota
burnsvillemn
cambridgeminnesota
carvercounty
chanhassen
chaska
coonrapids
cottagegrove
duluth
eaganmn
eastbethelmn
edenprairie
elkriver
grandrapidsmn
hastingsmn
hibbing
isantimn
kassonmantorville
mankato
MapleGroveMN
marshallmn
minneapolis
minnetonka
FargoMoorhead
morrismn
moundmn
moundsview
northfield
owatonna
plymouthmn
redwingmn
richfield
rochestermn
rochesterminnesota
rosemount
rosemountmn
saintbutt
saintbuttminnesota
saintpaul
stpaul
stpaulpark
saintpeter
shakopee
staples_minnesota
twincities
twincitiessocial
waconia
waterville
willmar
windommn
winon

phoenixville
PikeCountyPA
pittsburgh
poconos
pottsville
ReadingPA
rhawnhurst
Scranton
shaler
sharpsville
shippensburg
Springfielddelco
stroudsburg
towanda
trevose
upperstclair
VenangoCountyPA
WashingtonPA
westchesterpa
westmifflin
wexford
WilkesBarre
williamsport
wynnewood
Yardley
yorkpa
YorkCountyPA
rhodeisland
BlockIsland
Eastgreenwich
kentcountyri
misquamicut
narragansett
newport
pawtucket
providence
southcounty
southcarolina
AbbevilleSC
AndersonSC
andersoncounty
beaufort
CalhounFalls
charleston
clemson
columbiasc
danielisland
easley
EdgefieldSC
florence
fortmillsc
gtownsc
greenville
greenwoodsc
Greer
hiltonhead
kiawahisland
myrtlebeach
newberry
ninetysixsc
oconeesc
pawleysisland
Rockhill
spartanburg
spartanburgsc
summervillesc
upstatesc
wareshoals
southdakota
blackhills
TheBlackHills
madisonsd
mtrushmore
pierre
rapidcity
siouxfalls
tennessee
totallytennessee
athenstn
athensunofficial
bartlett
baylortn
chattanooga
clarksville
clevelandtn
Collierville
cookeville
crossville
crossville

FileNotFoundError: [Errno 2] No such file or directory: 'state_analysis/r.aberdeenwa.submission.json'

In [13]:
# with open('state_data_counts.pkl', 'wb') as file:
#     pickle.dump(state_data_counts, file)

# DEMOCRAT VS REPUBLICAN STATE

In [42]:
df_fed = pd.read_csv('federal_election.csv')[['year', 'state', 'party', 'candidatevotes', 'candidate']].dropna()
df_fed_2016 = df_fed[df_fed['year'] == 2016]
df_fed_2016 = df_fed_2016[df_fed_2016['party'].isin(['republican', 'democrat'])]
df_fed_2016['prop_republican'] = df_fed_2016['candidatevotes'] /df_fed_2016.groupby(by=['state'])['candidatevotes'].transform(sum)
rep_sorted = df_fed_2016[df_fed_2016['party'] == 'republican'].sort_values(by='prop_republican').iloc[2:]
least_rep = rep_sorted.iloc[:17]['state'].tolist()
most_rep = rep_sorted.iloc[50-17:]['state'].tolist()

# df_fed_win = df_fed_2016[idx][['state', 'party']]
# df_fed_win
# x = df_fed_win.groupby(by='state')['party'].apply(list)
# for key in x:
#     print(key)

In [24]:
def get_state_counts():
    state_to_region = pd.read_csv('state_to_region.csv')
    dates, epochs = get_dates(datetime.utcnow())
    counts = {}
    for i in range(1, 2):
        x = state_to_region.dropna().groupby(by=f'Region {i}')['State'].apply(list)
        for key in x.keys():
            print(key)
            region_data = []
            for state in x[key]:
                sub_count = {}
                state_data = get_subreddit_data(state)
#                 region_data.extend(state_data)
                times = []
                for ele in state_data:
                    times.append(ele['created_utc'])
                time_buckets = np.digitize(times, epochs)
                state_data = np.array(state_data)
                week_data = []
                for j in range(1, max(time_buckets)+1):
                    mask = np.where(time_buckets==j)
                    sub_count[j] =  len(state_data[mask])
                counts[state] = sub_count
    return counts
                    


In [32]:
state_counts = get_state_counts()
with open('state_counts.pkl', 'wb') as file:
    pickle.dump(state_counts, file)

Midwest
Mountain Plains
Northeast
Southeast
West Coast


In [ ]:
def subsample_data(state, state_data):
    

In [57]:
def get_region_data_pol():
#     df_fed = pd.read_csv('federal_election.csv')
#     df_fed_2016 = df_fed[df_fed['year'] == 2016]
#     idx = df_fed_2016.groupby(by='state')['candidatevotes'].transform(max) == df_fed_2016['candidatevotes']
#     df_fed_win = df_fed_2016[idx][['state', 'party']]
    dates, epochs = get_dates(datetime.utcnow())
#     dem_states = list(df_fed_win[df_fed_win['party'] == 'democrat']['state'])
#     rep_states = list(df_fed_win[df_fed_win['party'] == 'republican']['state'])
    df_fed = pd.read_csv('federal_election.csv')[['year', 'state', 'party', 'candidatevotes', 'candidate']].dropna()
    df_fed_2016 = df_fed[df_fed['year'] == 2016]
    df_fed_2016 = df_fed_2016[df_fed_2016['party'].isin(['republican', 'democrat'])]
    df_fed_2016['prop_republican'] = df_fed_2016['candidatevotes'] /df_fed_2016.groupby(by=['state'])['candidatevotes'].transform(sum)
    rep_sorted = df_fed_2016[df_fed_2016['party'] == 'republican'].sort_values(by='prop_republican').iloc[2:]
    least_rep = rep_sorted.iloc[:17]['state'].tolist()
    most_rep = rep_sorted.iloc[50-17:]['state'].tolist()


    x = {'Democratic': least_rep, 'Republican': most_rep}
    counts = {}
    for key in x.keys():
        print(key)
        region_data = []
        for state in x[key]:
            state_data = get_subreddit_data(state)
            state_data = subsample_data(state, state_data)
            region_data.extend(state_data)
        times = []
        for ele in region_data:
            times.append(ele['created_utc'])
        time_buckets = np.digitize(times, epochs)
        region_data = np.array(region_data)
        week_data = []
        for j in range(1, max(time_buckets)+1):
            mask = np.where(time_buckets==j)
            all_json = region_data[mask]
            text_data = [ele['body'] for ele in all_json if 'body' in ele.keys()]
            text_data.extend([ele['selftext'] for ele in all_json if 'selftext' in ele.keys()])
            with open(f"BaselineVoting/Week{j}.{key}.txt", "w", encoding="UTF-8") as region_date_file:
                count = 0
                for line in text_data:
                    count += 1
                    region_date_file.write(line)
                    region_date_file.write("\n")
                print(f"Week{j}.{key} COMPLETE!")
            week_data.append(count)
        counts[key] = week_data
#             with open(f"Region{i+1}/{key}.txt", "w") as file:
#                 print(x[key])
    return counts

In [58]:
counts = get_region_data_pol()

Democratic
Week1.Democratic COMPLETE!
Week2.Democratic COMPLETE!
Week3.Democratic COMPLETE!
Week4.Democratic COMPLETE!
Week5.Democratic COMPLETE!
Week6.Democratic COMPLETE!
Week7.Democratic COMPLETE!
Week8.Democratic COMPLETE!
Week9.Democratic COMPLETE!
Week10.Democratic COMPLETE!
Week11.Democratic COMPLETE!
Week12.Democratic COMPLETE!
Week13.Democratic COMPLETE!
Week14.Democratic COMPLETE!
Week15.Democratic COMPLETE!
Week16.Democratic COMPLETE!
Week17.Democratic COMPLETE!
Week18.Democratic COMPLETE!
Week19.Democratic COMPLETE!
Week20.Democratic COMPLETE!
Week21.Democratic COMPLETE!
Week22.Democratic COMPLETE!
Republican
Week1.Republican COMPLETE!
Week2.Republican COMPLETE!
Week3.Republican COMPLETE!
Week4.Republican COMPLETE!
Week5.Republican COMPLETE!
Week6.Republican COMPLETE!
Week7.Republican COMPLETE!
Week8.Republican COMPLETE!
Week9.Republican COMPLETE!
Week10.Republican COMPLETE!
Week11.Republican COMPLETE!
Week12.Republican COMPLETE!
Week13.Republican COMPLETE!
Week14.Republica

In [63]:
# political_data_diachronic = pd.DataFrame(counts).T
# political_data_diachronic['sum'] = political_data_diachronic.apply(sum, axis=1)
# political_data_diachronic.to_csv('political_diachronic_stats.csv')
for key in ['Democratic', 'Republican']:
    data = {}
    rel_files = [file for file in os.listdir('Voting') if file.endswith(f"{key}.txt")]
    for file in rel_files:
        period = file.find('.')
        data[int(file[4:period])] = open(f"Voting/{file}", 'r').readlines()
    pickle.dump(data, open(f"BaselineVoting/{key}.all_data.pkl", 'wb'))
    

# CULTURAL DIVISIONS
Sort into regions by date

In [20]:
def get_dates(latest_date):
    # Get all week increments from earliest date until present
    dates = []
    epochs = []
    curr = EARLIEST_DATE
    while curr < latest_date:
        dates.append(curr.strftime("%m/%d/%Y"))
        epochs.append(int(curr.timestamp()))
        curr += timedelta(weeks=1)
    dates.append(latest_date.strftime("%m/%d/%Y"))
    epochs.append(int(latest_date.timestamp()))
    return dates, epochs

In [21]:
def get_subreddit_data(subreddit):
    with open(f'{DIRNAME}{subreddit}.all_data.json', 'r') as file:
        data = file.readlines()
        return [json.loads(line) for line in data if line != ""] 

In [10]:
def get_region_data():
    state_to_region = pd.read_csv('state_to_region.csv')
    dates, epochs = get_dates(datetime.utcnow())
    counts = {}
    for i in range(1, 2):
        x = state_to_region.dropna().groupby(by=f'Region {i}')['State'].apply(list)
        print(i)
        sub_count = {}
        for key in x.keys():
            print(key)
            region_data = []
            for state in x[key]:
                state_data = get_subreddit_data(state)
                region_data.extend(state_data)
            times = []
            for ele in region_data:
                times.append(ele['created_utc'])
            time_buckets = np.digitize(times, epochs)
            region_data = np.array(region_data)
            week_data = []
            for j in range(1, max(time_buckets)+1):
                mask = np.where(time_buckets==j)
                all_json = region_data[mask]
                text_data = [ele['body'] for ele in all_json if 'body' in ele.keys()]
                text_data.extend([ele['selftext'] for ele in all_json if 'selftext' in ele.keys()])
#                 count = len(text_data)
#                 week_data.append(count)
#             sub_count[key] = week_data
#         counts[f'Region Mapping {i}'] = sub_count 
#     return counts
                with open(f"BaselineRegion{i}/Week{j}.{key}.txt", "w", encoding="UTF-8") as region_date_file:
                    for line in text_data:
                        
                        region_date_file.write(line)
                        region_date_file.write("\n")
                    print(f"Week{j}.{key} COMPLETE!")
#             with open(f"Region{i+1}/{key}.txt", "w") as file:
#                 print(x[key])

In [11]:
state_counts = get_region_data()

1
Midwest
Week1.Midwest COMPLETE!
Week2.Midwest COMPLETE!
Week3.Midwest COMPLETE!
Week4.Midwest COMPLETE!
Week5.Midwest COMPLETE!
Week6.Midwest COMPLETE!
Week7.Midwest COMPLETE!
Week8.Midwest COMPLETE!
Week9.Midwest COMPLETE!
Week10.Midwest COMPLETE!
Week11.Midwest COMPLETE!
Week12.Midwest COMPLETE!
Week13.Midwest COMPLETE!
Week14.Midwest COMPLETE!
Week15.Midwest COMPLETE!
Week16.Midwest COMPLETE!
Week17.Midwest COMPLETE!
Week18.Midwest COMPLETE!
Week19.Midwest COMPLETE!
Week20.Midwest COMPLETE!
Week21.Midwest COMPLETE!
Week22.Midwest COMPLETE!
Week23.Midwest COMPLETE!
Mountain Plains
Week1.Mountain Plains COMPLETE!
Week2.Mountain Plains COMPLETE!
Week3.Mountain Plains COMPLETE!
Week4.Mountain Plains COMPLETE!
Week5.Mountain Plains COMPLETE!
Week6.Mountain Plains COMPLETE!
Week7.Mountain Plains COMPLETE!
Week8.Mountain Plains COMPLETE!
Week9.Mountain Plains COMPLETE!
Week10.Mountain Plains COMPLETE!
Week11.Mountain Plains COMPLETE!
Week12.Mountain Plains COMPLETE!
Week13.Mountain Plain

In [42]:
for key in state_counts:
    state_region_df = pd.DataFrame(state_counts[key]).T
    state_region_df['sum'] = state_region_df.apply(sum, axis=1)
    state_region_df.to_csv(f'{key}.diachronic.csv')

In [6]:
# TODO FOR DISTRICT BY DISTRICT ANALYSIS
# Get rid of state full name
# get rid of state abbreviation
# Add space between county names
state_sub_map['Alabama']

['alabama',
 'Anniston',
 'ArabAlabama',
 'auburn',
 'baldwincountyal',
 'birmingham',
 'blountsville',
 'Calera',
 'CalhounCounty',
 'Cullman',
 'dothan',
 'DothanAL',
 'fairhopeAlabama',
 'FayetteCounty',
 'florenceal',
 'gadsden',
 'gordo',
 'greenvilleal',
 'Hartselle',
 'cityofhoover',
 'HooverAL',
 'huntsvillealabama',
 'JacksonvilleAL',
 'Madisonalabama',
 'marshallcountyal',
 'mobileal',
 'montgomery',
 'mountainbrook',
 'oba',
 'OxfordAL',
 'prattville',
 'Ragland',
 'selma',
 'shoals',
 '251',
 'SummerdaleAL',
 'sylacauga',
 'talladega',
 'ThomasvilleAL',
 'Trussville',
 'tuscaloosa',
 'vestavia',
 'vestaviahills']

# ORDER ALL POSTS BY DATE 